In [7]:
from flask import Flask, render_template, request
import pymongo as pm
import cv2
import numpy as np
from bson.binary import Binary
from six.moves import cPickle as pickle

In [32]:
name = 'mydb'
conn = pm.MongoClient('localhost:27017')
db = conn.get_database(name)

In [33]:
def insert_data(db_name, args_dict):
    '''
    db_name -> string i.e name of the db
    args_dict -> a dictionary of entries in db
    '''
    db[db_name].insert_one(args_dict)
    print('Data inserted successfully')

def read_data(db_name):
    '''
    returns a cursor of objects
    which can be iterated and printed
    '''
    cols = db[db_name].find({})
    return cols

#Update in data base
def update_data(db_name, idno, updation):
    '''
    db_name -> string
    idno -> id number of database entry in dict
    eg:- {'id':'02'}
    updation -> dict of elements to be updated
    eg:-{
        '$set':{
            'name':'Kevin11',
            'contact':'9664820165'
        }
    }
    '''
    db[db_name].update_one(idno, updation)
    print('Database updated successfully')

def delete_row(db_name, idno):
    '''
    Deletes the complete row 
    idno must be a dict {idno:'anything'}
    '''
    db[db_name].delete_many(idno)
    print('Row deleted')

In [34]:
delete_row(db_name=name, idno={'name':'dogs'})

Row deleted


In [46]:
app = Flask(__name__)

def write_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (100,100))
    return Binary(pickle.dumps(img, protocol=2))

def load_image_from_db(img_name, db_name):
    data = []
    db = conn.get_database(db_name)
    cols = read_data(db_name)
    for c in cols:
        print(c['name'])
        if c['name'] == img_name:
            x = pickle.loads(c['array'])
            data.append(x)
    return data
            
@app.route('/')
def select_file():
    return render_template('form.html')

@app.route('/upload', methods=['POST'])
def upload_in_db():
    preset_path = './images/'
    if request.method == 'POST':
        fname = request.form['name']
        fpath = request.form['imgfile']
        print(fname, fpath)
        args = {
            'name':fname,
            'array':write_image(preset_path + fpath)
        }
        insert_data(name, args_dict=args)
        return 'File uploaded'

In [43]:
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Mar/2018 21:14:44] "GET / HTTP/1.1" 200 -


dogs dogs.jpg


127.0.0.1 - - [11/Mar/2018 21:14:51] "POST /upload HTTP/1.1" 200 -


Data inserted successfully


In [47]:
data = load_image_from_db(img_name="dogs", db_name=name)

cat_black
dogs


In [48]:
cv2.imshow('Image', data[0])
cv2.waitKey(0)
cv2.destroyAllWindows()